In [5]:
import pickle
import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import plyfile
import numpy as np
import open3d as o3d
import h5py
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import open3d as o3d
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

import cv2
import numpy as np

import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import evaluation, scan3r


data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
#curr_scan_id =  "02b33e01-be2b-2d54-93fb-4145a709cec5"#"02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26" #"fcf66d88-622d-291c-871f-699b2d063630" #"fcf66d8a-622d-291c-8429-0e1109c6bb26"
curr_scan_id = scan3r.get_reference_id(data_dir,new_scan_id)
frame_number = "000007"
curr_frame_number = "000007"
new_frame_number = "000008"
patch_h= 18
image_height = 540
image_width = 960
patch_w = 32
patch_height = 30
patch_width = 30

/local/home/ekoller/BT


In [15]:
#visualitation of the following: with the new ids fill in an image with the new ids
#compute the resulting image when comparing stuff

def generate_pairs_pixel_level( dino_data,id_matches):
    
    new_ids = np.zeros((image_height, image_width))


    #go over every mask and fill in the id into the new_ids which it got mapped to
    for seg_region in dino_data:
        mask_id = seg_region["object_id"]
        
        #print("mask id ", mask_id)
        #get to what the region mapped in the embeddings
        matched_id = id_matches[str(mask_id)]
        #print("matched id ", matched_id)
        mask = seg_region["mask"]
        boolean_mask = mask == 225
        new_ids[boolean_mask] = matched_id #[0] is the id the second one is the error
        #visualize(boolean_mask, new_ids, f'Updated new_ids with mask id {mask_id}')

    #returns the new ids on a pixel wise level
    return new_ids

In [3]:
#this codesegment takes an image on a pixel wise level and quantizes it such that every patch has only the id of the most often occuring id
def quantize_to_patch_level(pixelwise_img):
    #get the shape of the pixelwise img
    input_h, input_w = pixelwise_img.shape
    patch_width = int(input_w/patch_w)
    patch_height= int(input_h/patch_h)

    patchwise_w = patch_w #number of patches
    patchwise_h = patch_h

    patchwise_id = np.zeros((patchwise_h,patchwise_w))

    for i in range(patchwise_h):
            for j in range(patchwise_w):
                # Define the coordinates of the current patch
                h_start = i * patch_height
                w_start = j * patch_width
                h_end = h_start + patch_height
                w_end = w_start + patch_width
                
                # Get the current patch from the input matrix
                patch = pixelwise_img[h_start:h_end, w_start:w_end]
                
                # get the most reoccuring id of the patch
                flattened_patch = patch.flatten()
                # Find the most common value in the patch
                value_counts = Counter(flattened_patch)
                most_common_id = value_counts.most_common(1)[0][0]
                
                # Assign the most common ID to the new matrix
                patchwise_id[i, j] = most_common_id


    return patchwise_id

In [4]:
#for a given scene get the colours of the differnt object_ids
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = plyfile.PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['red'], vertex['green'], vertex['blue'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color)
    
    return object_colors

colors = get_id_colours(data_dir,curr_scan_id)
print("cur scan id colors", colors.keys())
colors = get_id_colours(data_dir,new_scan_id)
print("new scan id colors", colors.keys())

cur scan id colors dict_keys([4, 18, 2, 0, 3, 1, 100, 16, 8, 9, 24, 25, 29, 6, 26, 10, 11, 23, 27, 22, 5, 20, 12, 15, 13, 14, 103, 21])
new scan id colors dict_keys([13, 20, 1, 4, 100, 6, 14, 15, 18, 29, 5, 8, 16, 105, 0])


In [5]:
#this function creates a color image of the size 960x540 from the patches
def create_color_img_from_obj_id(data_dir,scan_id,obj_id_mat):
    #access the mesh file to get the colour of the ids
    colour_dict = get_id_colours(data_dir, scan_id)
    #initialize the new matrix 
    og_height, og_width = obj_id_mat.shape
    new_height = og_height * patch_height
    new_width = og_width * patch_width

    colour_mat = np.zeros((new_height,new_width,3))

    #go over each element and assign the colour of the dictionary
    for h in range(og_height):
        for w in range(og_width):
            if obj_id_mat[h][w] in colour_dict.keys():
                colour = colour_dict[obj_id_mat[h][w]]
                colour_mat[h*patch_height:(h+1)*patch_height, w*patch_width:(w+1)*patch_width] = colour


    return colour_mat

In [6]:
#access the segmentation info

segmentation_info = {}
segmentation_info_path = osp.join("/media/ekoller/T7/Segmentation/DinoV2/objects", new_scan_id + ".h5")
with h5py.File(segmentation_info_path, 'r') as hdf_file:
             for frame_idx in hdf_file.keys():
                #init boxlist for curr frame
                bounding_boxes = []
                
                # get info 
                frame_group = hdf_file[frame_idx]
                
                #iterate over each boundingbox
                for bbox_key in frame_group.keys():
                    bbox_group = frame_group[bbox_key]
                    
                    #get te obj id
                    object_id = bbox_group.attrs['object_id']
                    
                    #get the boundingbox
                    bbox = bbox_group['bbox'][:]
                    
                    # get the maskt
                    mask = bbox_group['mask'][:]
                    
                    # append to list
                    bounding_boxes.append({
                        'object_id': object_id,
                        'bbox': bbox,
                        'mask': mask
                    })
                
                # stor it to the corresponding frame
                segmentation_info[frame_idx] = bounding_boxes

print("segmentation info", segmentation_info)
id_matches = {}
matches_path = osp.join("/local/home/ekoller/R3Scan/Predicted_Matches", new_scan_id + ".h5")
with h5py.File(matches_path, 'r') as hdf_file:
            id_matches = {}
            
            # Iterate through frame indices
            for frame_idx in hdf_file.keys():
                matches = {}
                
                # Access the group for each frame index
                frame_group = hdf_file[frame_idx]
                
                # Load the frame_id -> obj mappings
                for frame_id in frame_group.keys():
                    obj = frame_group[frame_id][()]
                    matches[frame_id] = int(obj)  # Convert back to int
                
                id_matches[frame_idx] = matches
                    



for frame, match_list in id_matches.items():
    # Extract and store the frame index and corresponding matches

    #assign each pixel to the new value, curr_scan_id , frame number is only for the size
    new_img_pixelwise = generate_pairs_pixel_level(segmentation_info[frame], match_list)

    #quantize to patches
    new_img_patchwise = quantize_to_patch_level(new_img_pixelwise)
    #aggregate to patches and colour it, we want the colours which are used in curr_scan id
    new_img_colour = create_color_img_from_obj_id(data_dir,curr_scan_id, new_img_patchwise)
    print("unique numbers in new img patchwise ", np.unique(new_img_patchwise))


    """
    access the gt of the image and get the accuracy of the prediction, print result and display 
    """
    #get the gt patches for the segmented scene! so for the dinov2 segmentation: but the colour will be based on the current scene
    gt_input_patchwise_path =  osp.join(data_dir,"files/patch_anno/patch_anno_32_18", new_scan_id + '.pkl')
    with open(gt_input_patchwise_path, 'rb') as file:
        gt_input_patchwise = pickle.load(file)
    #print("unique numbers in gt_input_colour ", np.unique(gt_input_patchwise[frame]))
    gt_input_colour = create_color_img_from_obj_id(data_dir,curr_scan_id,gt_input_patchwise[frame])


    # """
    # display the newly computed images next to each other
    # """

    #display the gt_image and the new patchwise image next to each other
    title1 = "gt_patces"
    title2 = "predicted_patches"


    # Create a blank canvas to combine images horizontally
    height = max(gt_input_colour.shape[0], new_img_colour.shape[0])  # Max height of both images
    width = gt_input_colour.shape[1] + new_img_colour.shape[1] + 20  # Total width of both images with a small gap
    combined_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Place images with titles on the blank canvas
    combined_image[:gt_input_colour.shape[0], :gt_input_colour.shape[1]] = gt_input_colour
    combined_image[:new_img_colour.shape[0], gt_input_colour.shape[1] + 20:] = new_img_colour

    # Add titles to the images
    cv2.putText(combined_image, title1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(combined_image, title2, (gt_input_colour.shape[1] + 30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Display the combined image
    cv2.imshow('Two Images Side by Side', combined_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



segmentation info {'000000': [{'object_id': 1, 'bbox': array([330, 390, 300, 150], dtype=int32), 'mask': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}, {'object_id': 2, 'bbox': array([480, 360, 180,  60], dtype=int32), 'mask': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}, {'object_id': 3, 'bbox': array([630,   0, 330, 420], dtype=int32), 'mask': array([[  0,   0,   0, ..., 225, 225, 225],
       [  0,   0,   0, ..., 225, 225, 225],
       [  0,   0,   0, ..., 225, 225, 225],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)}, {'object_id':

NameError: name 'generate_pairs_pixel_level' is not defined